<img src="https://i.imgur.com/6U6q5jQ.png"/>

<a target="_blank" href="https://colab.research.google.com/github/SocialAnalytics-StrategicIntelligence/TableOperations/blob/main/index.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

# Operations on Data Frames


Let me get the data on dengue from [Peru](https://www.datosabiertos.gob.pe/dataset/vigilancia-epidemiol%C3%B3gica-de-dengue):

In [4]:
import pandas as pd
linkData="https://github.com/SocialAnalytics-StrategicIntelligence/TableOperations/raw/main/dengue_ok.pkl"

dengue = pd.read_pickle(linkData)

# checking format
dengue.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 501236 entries, 0 to 501235
Data columns (total 9 columns):
 #   Column        Non-Null Count   Dtype   
---  ------        --------------   -----   
 0   departamento  501236 non-null  object  
 1   provincia     501236 non-null  object  
 2   distrito      501236 non-null  object  
 3   ano           501236 non-null  int64   
 4   semana        501236 non-null  int64   
 5   sexo          501236 non-null  object  
 6   edad          501236 non-null  int64   
 7   enfermedad    501236 non-null  category
 8   case          501236 non-null  int64   
dtypes: category(1), int64(4), object(4)
memory usage: 31.1+ MB


In [5]:
dengue.head()

,departamento,provincia,distrito,ano,semana,sexo,edad,enfermedad,case
0,HUANUCO,LEONCIO PRADO,LUYANDO,2000,47,M,9,SIN_SEÑALES,1
1,HUANUCO,LEONCIO PRADO,LUYANDO,2000,40,F,18,SIN_SEÑALES,1
2,HUANUCO,LEONCIO PRADO,JOSE CRESPO Y CASTILLO,2000,48,F,32,SIN_SEÑALES,1
3,HUANUCO,LEONCIO PRADO,JOSE CRESPO Y CASTILLO,2000,37,F,40,SIN_SEÑALES,1
4,HUANUCO,LEONCIO PRADO,MARIANO DAMASO BERAUN,2000,42,M,16,SIN_SEÑALES,1


In [6]:
# some exploration
dengue.describe().apply(lambda s: s.apply('{0:.5f}'.format))

,ano,semana,edad,case
count,501236.00000,501236.00000,501236.00000,501236.00000
mean,2014.77213,21.99838,28.96143,1.00000
std,6.14646,14.76658,18.15954,0.00000
min,2000.00000,1.00000,0.00000,1.00000
25%,2011.00000,11.00000,15.00000,1.00000
50%,2016.00000,18.00000,26.00000,1.00000
75%,2020.00000,32.00000,41.00000,1.00000
max,2022.00000,53.00000,106.00000,1.00000


Each row is a person:

In [7]:
dengue.head()

,departamento,provincia,distrito,ano,semana,sexo,edad,enfermedad,case
0,HUANUCO,LEONCIO PRADO,LUYANDO,2000,47,M,9,SIN_SEÑALES,1
1,HUANUCO,LEONCIO PRADO,LUYANDO,2000,40,F,18,SIN_SEÑALES,1
2,HUANUCO,LEONCIO PRADO,JOSE CRESPO Y CASTILLO,2000,48,F,32,SIN_SEÑALES,1
3,HUANUCO,LEONCIO PRADO,JOSE CRESPO Y CASTILLO,2000,37,F,40,SIN_SEÑALES,1
4,HUANUCO,LEONCIO PRADO,MARIANO DAMASO BERAUN,2000,42,M,16,SIN_SEÑALES,1


Let's start create some _info_ from these _data_!

# Average Age - by Year and Symptoms

## Aggregation

Having people, we need to count them by some grouping variable, in this case year (_ano_) and dengue status.


In [23]:
indexList=['ano','enfermedad']
aggregator={'edad': ['mean','median','std'], 'case':['sum']}
ByYear_stats=dengue.groupby(indexList,observed=True).agg(aggregator)
ByYear_stats

edad                     case
                       mean median        std    sum
ano  enfermedad                                     
2000 SIN_SEÑALES  28.412993   26.0  16.416231   5557
2001 SIN_SEÑALES  28.864195   26.0  16.887196  23276
     GRAVE        31.444000   28.0  17.428652    250
2002 SIN_SEÑALES  26.000991   23.0  15.380905   8071
     ALARMA       12.000000   12.0        NaN      1
     GRAVE        21.928571   19.0  12.651934     14
2003 SIN_SEÑALES  28.550690   26.0  16.019443   3334
     GRAVE        38.000000   30.0  15.061778     15
2004 SIN_SEÑALES  28.110901   25.0  16.495074   9513
     GRAVE        23.794118   19.0  15.069092     34
2005 SIN_SEÑALES  26.305244   24.0  16.160616   5625
     GRAVE        23.533333   23.0   9.775090     15
2006 SIN_SEÑALES  26.836237   24.0  15.954786   4018
     GRAVE        13.500000    8.5  15.842980      4
2007 SIN_SEÑALES  28.008401   25.0  16.267533   6309
     GRAVE        34.285714   30.0  20.901128     35
2008 SIN_SEÑALES  27.791540   25.0  16.865663  12789
     GRAVE        27.342857   22.0  17.055175     35
2009 SIN_SEÑALES  29.641535   27.0  17.324263  13396
     GRAVE        17.727273   13.0  12.610241     11
2010 SIN_SEÑALES  30.657988   28.0  17.554679  16669
     ALARMA       30.455224   27.0  18.319477    134
     GRAVE        29.512821   30.0  17.423323     39
2011 SIN_SEÑALES  24.114426   20.0  18.236159  24400
     ALARMA       24.430160   20.0  18.089344   3508
     GRAVE        28.181818   25.0  19.260349    176
2012 SIN_SEÑALES  25.204146   21.0  18.267408  23444
     ALARMA       23.926368   19.0  17.433246   4862
     GRAVE        25.085427   23.0  17.830791    199
2013 SIN_SEÑALES  27.805157   25.0  17.269582   9541
     ALARMA       26.058350   21.0  17.674662   3479
     GRAVE        29.680556   25.0  20.619850     72
2014 SIN_SEÑALES  28.384110   25.0  17.840599  12939
     ALARMA       26.247680   22.0  17.505294   4203
     GRAVE        29.652174   24.5  21.134023     92
2015 SIN_SEÑALES  30.470716   27.0  18.709214  29385
     ALARMA       31.158687   28.0  18.882697   6308
     GRAVE        35.691057   32.0  21.839253    123
2016 SIN_SEÑALES  30.646611   28.0  18.410342  21656
     ALARMA       30.173322   27.0  18.433588   3381
     GRAVE        32.048780   30.0  21.572924    123
2017 SIN_SEÑALES  32.828274   30.0  19.136399  60381
     ALARMA       32.446895   29.0  19.280475   7664
     GRAVE        39.568376   35.0  23.083484    234
2018 SIN_SEÑALES  29.790519   27.0  18.453384   3523
     ALARMA       30.257426   28.0  18.470292   1111
     GRAVE        30.250000   28.5  18.195586     64
2019 SIN_SEÑALES  27.896911   25.0  17.981282  12174
     ALARMA       29.475804   27.0  17.552928   2955
     GRAVE        34.215190   31.5  20.272457    158
2020 SIN_SEÑALES  28.295347   25.0  17.846855  41822
     ALARMA       26.811215   23.0  17.557003   5885
     GRAVE        30.840000   26.0  20.213176    225
2021 SIN_SEÑALES  28.832612   26.0  17.674078  40881
     ALARMA       28.537688   26.0  17.915630   3781
     GRAVE        32.682171   27.0  21.384495    129
2022 SIN_SEÑALES  29.486843   27.0  18.555726  55293
     ALARMA       27.962771   24.0  18.813110   7709
     GRAVE        35.146226   32.0  23.158476    212

In [10]:
ByYear_stats.sort_values(('edad','median'),ascending=False)

edad          case
                       mean median    sum
ano  enfermedad                          
2017 GRAVE        39.568376   35.0    234
2015 GRAVE        35.691057   32.0    123
2022 GRAVE        35.146226   32.0    212
2019 GRAVE        34.215190   31.5    158
2007 GRAVE        34.285714   30.0     35
2016 GRAVE        32.048780   30.0    123
2003 GRAVE        38.000000   30.0     15
2010 GRAVE        29.512821   30.0     39
2017 SIN_SEÑALES  32.828274   30.0  60381
     ALARMA       32.446895   29.0   7664
2018 GRAVE        30.250000   28.5     64
2015 ALARMA       31.158687   28.0   6308
2001 GRAVE        31.444000   28.0    250
2016 SIN_SEÑALES  30.646611   28.0  21656
2018 ALARMA       30.257426   28.0   1111
2010 SIN_SEÑALES  30.657988   28.0  16669
2016 ALARMA       30.173322   27.0   3381
2010 ALARMA       30.455224   27.0    134
2022 SIN_SEÑALES  29.486843   27.0  55293
2009 SIN_SEÑALES  29.641535   27.0  13396
2021 GRAVE        32.682171   27.0    129
2018 SIN_SEÑALES  29.790519   27.0   3523
2019 ALARMA       29.475804   27.0   2955
2015 SIN_SEÑALES  30.470716   27.0  29385
2001 SIN_SEÑALES  28.864195   26.0  23276
2020 GRAVE        30.840000   26.0    225
2021 SIN_SEÑALES  28.832612   26.0  40881
     ALARMA       28.537688   26.0   3781
2000 SIN_SEÑALES  28.412993   26.0   5557
2003 SIN_SEÑALES  28.550690   26.0   3334
2013 SIN_SEÑALES  27.805157   25.0   9541
2019 SIN_SEÑALES  27.896911   25.0  12174
2007 SIN_SEÑALES  28.008401   25.0   6309
2008 SIN_SEÑALES  27.791540   25.0  12789
2020 SIN_SEÑALES  28.295347   25.0  41822
2004 SIN_SEÑALES  28.110901   25.0   9513
2011 GRAVE        28.181818   25.0    176
2014 SIN_SEÑALES  28.384110   25.0  12939
2013 GRAVE        29.680556   25.0     72
2014 GRAVE        29.652174   24.5     92
2005 SIN_SEÑALES  26.305244   24.0   5625
2022 ALARMA       27.962771   24.0   7709
2006 SIN_SEÑALES  26.836237   24.0   4018
2005 GRAVE        23.533333   23.0     15
2002 SIN_SEÑALES  26.000991   23.0   8071
2020 ALARMA       26.811215   23.0   5885
2012 GRAVE        25.085427   23.0    199
2008 GRAVE        27.342857   22.0     35
2014 ALARMA       26.247680   22.0   4203
2013 ALARMA       26.058350   21.0   3479
2012 SIN_SEÑALES  25.204146   21.0  23444
2011 SIN_SEÑALES  24.114426   20.0  24400
     ALARMA       24.430160   20.0   3508
2004 GRAVE        23.794118   19.0     34
2002 GRAVE        21.928571   19.0     14
2012 ALARMA       23.926368   19.0   4862
2009 GRAVE        17.727273   13.0     11
2002 ALARMA       12.000000   12.0      1
2006 GRAVE        13.500000    8.5      4

In [11]:
ByYear_stats.describe()

edad                     case
            mean     median           sum
count  59.000000  59.000000     59.000000
mean   28.423533  25.084746   8495.525424
std     4.750027   4.713473  13457.478926
min    12.000000   8.500000      1.000000
25%    26.558230  23.000000    126.000000
50%    28.550690  26.000000   3508.000000
75%    30.558663  28.000000   9527.000000
max    39.568376  35.000000  60381.000000

In [12]:
ByYear_stats.columns

MultiIndex([('edad',   'mean'),
            ('edad', 'median'),
            ('case',    'sum')],
           )

In [13]:
# For easier manipualtion outside Python, we could flatten the index hierarchy:
["_".join(name) for name in ByYear_stats.columns]

['edad_mean', 'edad_median', 'case_sum']

In [14]:
# changing
ByYear_stats.columns=["_".join(name) for name in ByYear_stats.columns]
ByYear_stats.reset_index(drop=False,inplace=True)
# get:
ByYear_stats

,ano,enfermedad,edad_mean,edad_median,case_sum
0,2000,SIN_SEÑALES,28.412993,26.0,5557
1,2001,SIN_SEÑALES,28.864195,26.0,23276
2,2001,GRAVE,31.444000,28.0,250
3,2002,SIN_SEÑALES,26.000991,23.0,8071
4,2002,ALARMA,12.000000,12.0,1
5,2002,GRAVE,21.928571,19.0,14
6,2003,SIN_SEÑALES,28.550690,26.0,3334
7,2003,GRAVE,38.000000,30.0,15
8,2004,SIN_SEÑALES,28.110901,25.0,9513
9,2004,GRAVE,23.794118,19.0,34


In [15]:
ByYear_stats['enfermedad_text']=ByYear_stats.enfermedad.astype(str)

ByYear_stats.replace({'enfermedad_text':{'SIN_SEÑALES':'1_SIN_SEÑALES','ALARMA':'2_ALARMA','GRAVE':'3_GRAVE'}},inplace=True)

In [22]:
import altair as alt

alt_data=alt.Chart(ByYear_stats)
enc_data=alt_data.encode(alt.X('ano:O'),
                         alt.Y('edad_mean:Q'),
                         alt.Color('enfermedad_text:N'),tooltip=['ano']).interactive()
enc_data.mark_line() + enc_data.mark_errorband()

alt.LayerChart(...)

In [149]:
enc_data=alt_data.encode(alt.X('ano:O'),
                         alt.Y('edad_mean:Q'),
                        alt.Size('case_sum:Q'),
                         alt.Color('enfermedad:N'),tooltip=['case_sum']).interactive()
enc_data.mark_point()

alt.Chart(...)

In [156]:
enc_data=alt_data.encode(alt.X('ano:O'),
                         alt.Y('edad_mean:Q'),
                         alt.Color('enfermedad_text:N'),tooltip=['case_sum']).interactive()
enc_data.mark_bar()

alt.Chart(...)

In [157]:
enc_data=alt_data.encode(alt.X('ano:O'),
                         alt.Column('enfermedad_text:N'),
                         alt.Y('edad_mean:Q'),
                         alt.Color('enfermedad_text:N'),tooltip=['case_sum']).interactive()
enc_data.mark_rule()

alt.Chart(...)

## Reshaping

Notice the variables are in three columns: **edad_mean** /	**edad_median** / 	**case_sum**. We could reshape those columns to a long format: 

In [ ]:
theVarsAsIndex=['ano','enfermedad']

# stacking  and resetting index
ByYear_LongStats=ByYear_stats.set_index(theVarsAsIndex).stack().reset_index()

#result
ByYear_LongStats

In [ ]:
# just renaming
ByYear_LongStats.rename(columns={'level_2':'statsName',0:'statsValue'},inplace=True)
ByYear_LongStats

In [ ]:
# still ordinal?
ByYear_LongStats.enfermedad.dtype

In [ ]:
# NO csv, PICKLE!
ByYear_LongStats.to_pickle('dataFiles/ByYear_LongStats.pkl')

# Dengue by Location (Province)

## Aggregating

We can redo the previous process, adding _departamento_ and _province_: 

In [ ]:
indexList=['ano','departamento','provincia','enfermedad']
aggregator={'case':['sum']}
ByYearPlace=dengue.groupby(indexList,observed=True).agg(aggregator)
ByYearPlace

Before flattening the output data frame in long format, you could create a wide shape:

## Long to wide

In [ ]:
#simply
ByYearPlace.unstack()

In [ ]:
# a more familiar look
ByYearPlace_wide=ByYearPlace.unstack().reset_index()
ByYearPlace_wide

In [ ]:
# zero instead of missing
ByYearPlace_wide.fillna(0,inplace=True)
ByYearPlace_wide

In [ ]:
# you expected
ByYearPlace_wide.columns

In [ ]:
#prepare
["_".join(names) if names[1]!='' else names[0] for names in ByYearPlace_wide.columns]

In [ ]:
# change
ByYearPlace_wide.columns=["_".join(names) if names[1]!='' else names[0] for names in ByYearPlace_wide.columns]
ByYearPlace_wide

What about finding the _provincia_ most affected in a _departamento_?

In [ ]:
where = ByYearPlace_wide.groupby(['ano','departamento'])['case_sum_ALARMA'].idxmax()
worst_prov_year = ByYearPlace_wide.loc[where].reset_index(drop=True)
worst_prov_year

In [ ]:
# amount of worst provinces per region
len(worst_prov_year.provincia.value_counts())

In [ ]:
# amount of worst provinces per region - cleaner
len(worst_prov_year[worst_prov_year.case_sum_ALARMA>0].provincia.value_counts())

## Filtering

Lets' filter some rows based on what we just computed:

In [ ]:
worst_ProvYear_alarma=worst_prov_year[worst_prov_year.case_sum_ALARMA>0].loc[:,['departamento','provincia']]
worst_ProvYear_alarma.reset_index(drop=True,inplace=True)
worst_ProvYear_alarma

In [ ]:
# adding a column of ones
worst_ProvYear_alarma['case']=1
worst_ProvYear_alarma

## Frequency table

With filtered data, let's create a crosstabulation:

In [ ]:
indexList=['departamento','provincia']
aggregator={'case':['sum']}
worst_ProvYear_alarma_Frequency=worst_ProvYear_alarma.groupby(indexList,observed=True).agg(aggregator)
worst_ProvYear_alarma_Frequency

In [ ]:
# flattening with counts
worst_ProvYear_alarma_Frequency.reset_index()

In [ ]:
# final look
worst_ProvYear_alarma_Frequency.columns=['case']
worst_ProvYear_alarma_Frequency.reset_index(inplace=True)
worst_ProvYear_alarma_Frequency

Saving the frequencies in a file:

In [ ]:
worst_ProvYear_alarma_Frequency.to_csv('dataFiles/worst_ProvYear_alarma_Frequency.csv',index=False)

# The 'ALARM' symptoms level

## Aggregating

Let's check a previous data frame:

In [ ]:
ByYearPlace_wide

This time, I want two variables:

In [ ]:
indexList=['ano','departamento']
aggregator={'case_sum_SIN_SEÑALES':['sum'],'case_sum_ALARMA':['sum']}
ByYearPlace=ByYearPlace_wide.groupby(indexList,observed=True).agg(aggregator)
ByYearPlace.columns=['sum_SIN_SEÑALES','sum_ALARMA']
ByYearPlace.reset_index(inplace=True)
ByYearPlace

## Creating information

I will create a new variable:

In [ ]:
ByYearPlace['rateAlarma']=(ByYearPlace['sum_ALARMA']/ByYearPlace['sum_SIN_SEÑALES'])
ByYearPlace['rateAlarma'].describe()

We got _inf_ values:

In [ ]:
import numpy as np #identify with numpy
ByYearPlace[np.isinf(ByYearPlace.rateAlarma)]

We need to make a decision. I did this:

In [ ]:
ByYearPlace.loc[186,'rateAlarma']=1
ByYearPlace.drop(columns=['sum_SIN_SEÑALES','sum_ALARMA'],inplace=True)
ByYearPlace['rateAlarma'].describe()

### Discretizing

Sometimes you need a numerical variable as an ordinal variable: 

In [ ]:
edges=[-1, .1, .25, .5,.75,1,2]
theLabels=["less10%","10-25%","25-50","51-75%","75-100%","above100%"]
ByYearPlace["rateAlarma.cut"]=pd.cut(ByYearPlace['rateAlarma'], include_lowest=True,
                                     bins=edges, 
                                     labels=theLabels,
                                     ordered=True)

In [ ]:
# we have
ByYearPlace

We could check the yearly behavior:

In [ ]:
ByYearPlace.groupby('ano').describe()

Let's do some **filtering**:

In [ ]:
ByYearPlace=ByYearPlace[ByYearPlace.ano>=2012]
ByYearPlace.reset_index(drop=True,inplace=True)
ByYearPlace

In [ ]:
ByYearPlace.info()

In [ ]:
# the category should be exported as pickle

ByYearPlace.to_pickle("dataFiles/ByYearPlace.pkl")